In [ ]:
%matplotlib inline
import matplotlib 
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

import os

import cv2

DETECTIONS_A = "/Users/vgenty/Desktop/rcnn/alex_detections/detections_5/"
DETECTIONS_G = "/Users/vgenty/Desktop/rcnn/detections_5/"
VALID = "/Users/vgenty/Desktop/rcnn/valid/"
dets_A = os.listdir(DETECTIONS_A)
dets_G = os.listdir(DETECTIONS_G)

In [ ]:
idx_to_class = {1 : 'eminus',
                2 : 'proton',
                3 : 'pizero',
                4 : 'muminus',
                5 : 'gamma'}

In [ ]:
results_A = {'eminus' : [], 'proton' : [], 'pizero' : [], 'muminus' : [], 'gamma' : []}
results_G = {'eminus' : [], 'proton' : [], 'pizero' : [], 'muminus' : [], 'gamma' : []}

for ix, det in enumerate(dets_A):
    d_truth  = det.split("_")[-1][:-4]
    dets_df  = pd.read_csv(DETECTIONS_A+det,names=['prob','xmin','ymin','xmax','ymax'])
    top      = dets_df.sort(columns='prob',ascending=False).iloc[0]
    results_A[d_truth].append({'name':top.name,'prob':top.prob})

for ix, det in enumerate(dets_G):
    d_truth  = det.split("_")[-1][:-4]
    dets_df  = pd.read_csv(DETECTIONS_G+det,names=['prob','xmin','ymin','xmax','ymax'])
    top      = dets_df.sort(columns='prob',ascending=False).iloc[0]
    results_G[d_truth].append({'name':top.name,'prob':top.prob})

In [ ]:
classifieds_A = { 'eminus' : {},
                'proton' : {},
                'pizero' : {},
                'muminus': {},
                'gamma' : {}}

classifieds_G = { 'eminus' : {},
                'proton' : {},
                'pizero' : {},
                'muminus': {},
                'gamma' : {}}


for thresh in np.arange(0,1.01,0.01):
    for particle in classifieds_A :
        par = results_A[particle]
        for res in par:
            
            if float(res['prob'] < thresh):
                continue
                
            if thresh not in classifieds_A[particle]:
                classifieds_A[particle][thresh] = [0 for _ in xrange(5)]
                
            classifieds_A[particle][thresh][res['name'] - 1] += 1

for thresh in np.arange(0,1.01,0.01):
    for particle in classifieds_G :
        par = results_G[particle]
        for res in par:
            
            if float(res['prob'] < thresh):
                continue
                
            if thresh not in classifieds_G[particle]:
                classifieds_G[particle][thresh] = [0 for _ in xrange(5)]
                
            classifieds_G[particle][thresh][res['name'] - 1] += 1


p_df_A = pd.DataFrame(classifieds_A)
p_df_G = pd.DataFrame(classifieds_G)

In [ ]:
matplotlib.rcParams['font.size'] = 16

for par in results_A:
    
    thresh_0_sum_A =  float(np.array(p_df_A[par].ix[0.0]).sum())
    thresh_0_sum_G =  float(np.array(p_df_G[par].ix[0.0]).sum())

    fig,ax = plt.subplots(figsize=(10,6))
    
    others_A = np.zeros(p_df_A[par].iloc[:].str[i].values.shape)
    others_G = np.zeros(p_df_G[par].iloc[:].str[i].values.shape)
    

    for i in xrange(5):
        if idx_to_class[i+1] == par:
            ax.plot(p_df_A.index.values,
                    (p_df_A[par].iloc[:].str[i].values / thresh_0_sum_A)*100.0,
                    '-',lw='2',
                    label="Alex: {}".format(idx_to_class[i+1]))

            ax.plot(p_df_G.index.values,
                    (p_df_G[par].iloc[:].str[i].values / thresh_0_sum_G)*100.0,
                    '-',lw='2',
                    label="Google: {}".format(idx_to_class[i+1]),color='red')
            continue

        others_A+= (p_df_A[par].iloc[:].str[i].values / thresh_0_sum_A)*100.0
        others_G+= (p_df_G[par].iloc[:].str[i].values / thresh_0_sum_G)*100.0
       
    ax.plot(p_df_A.index.values,
            others_A,
            '-',lw='2',
            label="All Others Alex",color='black')

    ax.plot(p_df_G.index.values,
            others_G,
            '-',lw='2',
            label="All Others Google",color='gray')

    ax.set_ylabel('Classified % by RCNN'.format(par),fontweight='bold')
    ax.set_xlabel('Detection prob',fontweight='bold')
    ax.set_ylim(0,100.0)
    ax.legend(loc='best',fontsize=12)
    ax.set_title("Truth: {}\nSample Size: {}".format(par,thresh_0_sum))
    plt.savefig('eff_5_class_{}_compare.pdf'.format(par), format='pdf', dpi=1000)
    plt.show()

In [ ]:
import matplotlib.gridspec as gridspec

matplotlib.rcParams['font.size'] = 16
matplotlib.rcParams['font.family'] = 'serif'
for par in results_A:
    
    sums_A = np.array([float(np.sum(j)) for j in p_df_A[par].values])
    sums_G = np.array([float(np.sum(j)) for j in p_df_G[par].values])
    fig,_ = plt.subplots(figsize=(16,8))
    
    gs = gridspec.GridSpec(2, 1,
                           width_ratios=[1,1],
                           height_ratios=[1,2])
    
    ax = plt.subplot(gs[0])
    ax.plot(p_df_A.index.values,sums_A,'-',color='black',lw=2,label="Alex")
    ax.plot(p_df_G.index.values,sums_G,'--',color='black',lw=2,label="Google")
    ax.legend(loc='best',fontsize=12)
    ax.set_ylabel('N Images'.format(par),fontweight='bold')
    ax.set_title("Purity of truth: {}\n".format(par))
    ax.set_xticklabels([])



    ax = plt.subplot(gs[1])
    
    others_A = np.zeros(p_df_A[par].iloc[:].str[i].values.shape)
    others_G = np.zeros(p_df_G[par].iloc[:].str[i].values.shape)
    

    for i in xrange(5):
        if idx_to_class[i+1] == par:
            ax.plot(p_df_A.index.values,
                    (p_df_A[par].iloc[:].str[i].values / sums_A)*100.0,
                    '-',lw='2',
                    label="Alex: {}".format(idx_to_class[i+1]))

            ax.plot(p_df_G.index.values,
                    (p_df_G[par].iloc[:].str[i].values / sums_G)*100.0,
                    '-',lw='2',
                    label="Google: {}".format(idx_to_class[i+1]),color='red')
            continue

        others_A+= (p_df_A[par].iloc[:].str[i].values / sums_A)*100.0
        others_G+= (p_df_G[par].iloc[:].str[i].values / sums_G)*100.0
       
    ax.plot(p_df_A.index.values,
            others_A,
            '-',lw='2',
            label="All Others Alex",color='black')

    ax.plot(p_df_G.index.values,
            others_G,
            '-',lw='2',
            label="All Others Google",color='gray')
    
    ax.set_ylabel('Classified % by RCNN'.format(par),fontweight='bold')
    ax.set_xlabel('Detection prob treshold',fontweight='bold')
   
    ax.legend(loc='best',fontsize=12)
    plt.savefig('purity_5_class_{}_compare.pdf'.format(par), format='pdf', dpi=1000)
    ax.set_ylim(0,100)
    plt.tight_layout()
    plt.show()